# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [80]:
import os
import pandas as pd
import numpy as np
import freesound
import essentia
import essentia.standard as estd
from IPython.display import display

FREESOUND_API_KEY = 'cLgekQJZGnROViU6X1JQhCatkEFB5TuaopM0mqDP'  # My Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store
TARGET_PATH = '126095__xserra__istanbul-sazendeleri-6.wav'

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [81]:
# Define some util functions

def query_freesound(query, filter, sort, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results,
        sort = sort,
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

def get_key_from_target(target_path):
    loader = estd.MonoLoader(filename=target_path)
    audio = loader()
    key_algo = estd.KeyExtractor()
    key, scale, key_strength = key_algo(audio)
    return '{0} {1}'.format(key, scale)

In [82]:
# Build our collection of sounds

key = get_key_from_target(TARGET_PATH)
print(key)
# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'guitar',
        'filter': 'ac_tonality:"{0}" ac_tonality_confidence:[0.9 TO 1.0]'.format(key),
        'num_results': 20,
        'sort': 'downloads_desc',
    },
    {
        'query': 'saxaphone',
        'filter': 'ac_tonality:"{0}" ac_tonality_confidence:[0.9 TO 1.0]'.format(key),
        'num_results': 20, 
        'sort': 'downloads_desc',
    },
    {
        'query': 'violin',
        'filter': 'ac_tonality:"{0}" ac_tonality_confidence:[0.9 TO 1.0]'.format(key),
        'num_results': 20, 
        'sort': 'downloads_desc',
    },
    {
        'query': 'wilhelm',
        'filter': None,
        'num_results': 10, 
        'sort': 'score',
    },
    {
        'query': 'dog bark',
        'filter': None,
        'num_results': 10, 
        'sort': 'downloads_desc',
    },
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['sort'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

D minor
Saved DataFrame with 41 entries! dataframe.csv


,freesound_id,license,name,path,tags,username
0,218801,http://creativecommons.org/licenses/by/3.0/,Acoustic guitar - country,files/218801_3908940-hq.ogg,"[solo, atmospheric, country, guitar, music, tu...",ValentinSosnitskiy
1,15834,http://creativecommons.org/licenses/sampling+/...,synth_guitar_129_6_bpm.wav,files/15834_37597-hq.ogg,"[guitar, metal, riff, synth, synthesizer]",KrimZon
2,244022,http://creativecommons.org/publicdomain/zero/1.0/,Victory (Rock Guitar Tapping),files/244022_2979758-hq.ogg,"[Rock, Metal, Guitar, Riff, Tapping, Distorted]",DeathToMayo
3,13771,http://creativecommons.org/licenses/by/3.0/,blues.loop02.distort.fast.wav,files/13771_8043-hq.ogg,"[blues, distortion, electric-guitar, loop, mus...",dobroide
4,13770,http://creativecommons.org/licenses/by/3.0/,blues.loop02. distort.wav,files/13770_8043-hq.ogg,"[blues, distortion, electric-guitar, loop, mus...",dobroide
5,4266,http://creativecommons.org/licenses/by/3.0/,Dhammer.wav,files/4266_4948-hq.ogg,"[electric, guitar, hammers, solo]",NoiseCollector
6,58305,http://creativecommons.org/licenses/by-nc/3.0/,tk9 RIFF 5E REVRS.WAV,files/58305_215874-hq.ogg,"[acoustic-guitar, blues, guitar, music, riff]",ERH
7,388100,http://creativecommons.org/publicdomain/zero/1.0/,Distorted Synth Atmoslead.wav,files/388100_5829171-hq.ogg,"[arp, atmosphere, loops, ambient, arpeggiator,...",furbyguy
8,120988,http://creativecommons.org/licenses/by/3.0/,120 disto.wav,files/120988_2193266-hq.ogg,"[drum, drums, loop, loops]",Thirsk
9,340087,http://creativecommons.org/licenses/by/3.0/,Metal Backing Track.wav,files/340087_4162634-hq.ogg,"[Backing-Track, Guitar, Power-Metal, Distorted...",thunderstorm10
